# Random Forest Model
### Loan Default Prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, classification_report

from preprocessing import load_and_clean_data, preprocess_features

## 1. Load and Prepare Data

In [ ]:
df = load_and_clean_data('loan_data.csv')
print(f'Shape after cleaning: {df.shape}')
df.head()

In [ ]:
df = preprocess_features(df)

X = df.drop(columns=['loan_status'])
y = df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Train set: {X_train.shape}')
print(f'Test set: {X_test.shape}')

## 2. Train the Model

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=10,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)
print('Model trained.')

## 3. Evaluate the Model

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1]

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_prob)

print(f'Train Accuracy: {train_acc * 100:.2f}%')
print(f'Test Accuracy:  {test_acc * 100:.2f}%')
print(f'ROC-AUC Score:  {roc_auc:.4f}')

In [ ]:
print('Classification Report:')
print(classification_report(y_test, y_test_pred, target_names=['No Default', 'Default']))

## 4. Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

fig, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Default', 'Default'],
            yticklabels=['No Default', 'Default'], ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix - Random Forest')
plt.tight_layout()
plt.show()

## 5. ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Baseline')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve - Random Forest')
ax.legend()
plt.tight_layout()
plt.show()

## 6. Feature Importance

In [ ]:
feature_names = list(X_train.columns)
importances = model.feature_importances_

feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feat_df = feat_df.sort_values('Importance', ascending=True)

fig, ax = plt.subplots(figsize=(8, 6))
ax.barh(feat_df['Feature'], feat_df['Importance'], color='steelblue')
ax.set_xlabel('Importance')
ax.set_title('Random Forest - Feature Importances')
plt.tight_layout()
plt.show()

print('\nTop 10 most important features:')
print(feat_df.sort_values('Importance', ascending=False).head(10).to_string(index=False))